Exemple d'url à tester : 
- https://www.zoom.us/fr/signup?ampDeviceId=59043737-0e3c-412d-af6b-91f85e29b81f&ampSessionId=1766618944644&_ics=1766618944230&irclickid=%7E16Z5-c5Z4-2451W023S0QHIypgmnkmdiea723SUSICxqpnea841R&_gl=1*tveo27*_gcl_au*MTc4NDg4MjQ0Ny4xNzY2NjE4OTQz#/signup
- https://maratp.ratp.fr/newsletter/
- https://www.facebook.com/r.php?entry_point=login&locale=fr_FR
- https://jobs.smartrecruiters.com/oneclick-ui/company/Devoteam/publication/08190f81-022c-4a32-94c9-b496b29d05a7?dcr_ci=Devoteam&sid=306448a8-a471-41bb-b86a-218ae712f877

In [23]:
import random
import time

import requests
from bs4 import BeautifulSoup

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as ec
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager


## Test de récupération de la pge html brut.

In [24]:
TIMEOUT = 15
USER_AGENTS = [
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36",
    "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/33.0.1750.517 Safari/537.36",
    "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36",
]

In [30]:
#Les fonctions qui servent à l'extraction du html brut.

def create_driver() -> webdriver.Chrome:
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--disable-gpu")
    options.add_argument("--no-sandbox")

    return webdriver.Chrome(
        options=options,
        service=Service(ChromeDriverManager().install())
    )


# Cas des pages avec du JavaScript dynamique (formulaire non accessible avec le code source) .
def load_main_page(driver: webdriver.Chrome, url: str, wait_seconds: int) -> str:
    driver.get(url)
    WebDriverWait(driver, wait_seconds).until(
        ec.presence_of_element_located((By.TAG_NAME, "body")))
    return driver.page_source


# Cas avec iframes.
def get_iframes(driver: webdriver.Chrome) -> list:
    return driver.find_elements(By.TAG_NAME, "iframe")


def load_iframe_html(driver: webdriver.Chrome, iframe) -> str:
    driver.switch_to.frame(iframe)
    html = driver.page_source
    driver.switch_to.default_content()
    return html


def fetch_html_with_selenium(url: str, wait_seconds: int = 10) -> str:
    driver = create_driver()

    try:
        main_html = load_main_page(driver, url, wait_seconds)

        # Cas simple : formulaire dans le DOM principal
        if "<form" in main_html.lower():
            return main_html

        # Recherche éventuelle dans les iframes
        iframes = get_iframes(driver)

        for iframe in iframes:
            iframe_html = load_iframe_html(driver, iframe)

            if "<form" in iframe_html.lower():
                return iframe_html

        return main_html

    finally:
        driver.quit()


In [31]:
url = "https://maratp.ratp.fr/newsletter/"
print(fetch_html_with_selenium(url))

<html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns="http://www.w3.org/1999/xhtml" lang="fr" style="height: auto;"><head>
        <meta charset="utf-8">
        
            <title>STUDIO - Landing Page RATP</title>
        
        













        
    







    <meta name="robots" content="noindex,nofollow">

    <meta name="viewport" content="width=device-width, initial-scale=1">

    
    


    
    

    
    <script src="https://files.qualifio.com/kit/elastic-apm-opentracing.umd.min.js" async=""></script><script async="" src="https://files.qualifio.com/kit/qualp.2.min.js?v=20231116"></script><script async="" src="https://files.qualifio.com/kit/qualp.2.min.js?v=20231116"></script><script type="text/javascript" src="https://files.qualifio.com/static/common/js/jquery/1.11.3/jquery.min.js?v=1"></script>

    
    <script type="text/javascript" src="https://files.qualifio.com/static/common/js/jquery/jquery-migrate-1.2.1.min.js?v=1"></script>

    
    
        <link rel=

In [33]:
url = "https://maratp.ratp.fr/newsletter/"

headers = {
    "User-Agent": random.choice(USER_AGENTS),
    "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8",
    "Accept-Language": "fr-FR,fr;q=0.9,en-US;q=0.8,en;q=0.7",
    "Connection": "close",
}


try:
    response = requests.get(url, headers=headers, timeout=TIMEOUT)
    response.raise_for_status()
    html = response.text
    html_lower = html.lower()
    limit_size = 1000
    if len(html) < limit_size or "<form" not in html_lower:
        html = fetch_html_with_selenium(url)
    print("Pas d'erreur : ", response.status_code, html)

except requests.RequestException:
    html = fetch_html_with_selenium(url)
    print("Avec erreur: ", 200, html)


Pas d'erreur :  200 <html xmlns:fb="http://www.facebook.com/2008/fbml" xmlns="http://www.w3.org/1999/xhtml" lang="fr" style="height: auto;"><head>
        <meta charset="utf-8">
        
            <title>STUDIO - Landing Page RATP</title>
        
        













        
    







    <meta name="robots" content="noindex,nofollow">

    <meta name="viewport" content="width=device-width, initial-scale=1">

    
    


    
    

    
    <script src="https://files.qualifio.com/kit/elastic-apm-opentracing.umd.min.js" async=""></script><script async="" src="https://files.qualifio.com/kit/qualp.2.min.js?v=20231116"></script><script async="" src="https://files.qualifio.com/kit/qualp.2.min.js?v=20231116"></script><script type="text/javascript" src="https://files.qualifio.com/static/common/js/jquery/1.11.3/jquery.min.js?v=1"></script>

    
    <script type="text/javascript" src="https://files.qualifio.com/static/common/js/jquery/jquery-migrate-1.2.1.min.js?v=1"></script>

    
   

## Test de la detection des fomulaires

In [34]:
def has_form_tag(soup: BeautifulSoup) -> bool:
    return len(soup.find_all("form")) > 0


def count_form_tags(soup: BeautifulSoup) -> int:
    return len(soup.find_all("form"))


def has_input_fields(soup: BeautifulSoup) -> bool:
    fields = soup.find_all(["input", "select", "textarea"])
    return len(fields) > 0


def has_action_button(soup: BeautifulSoup) -> bool:
    buttons = soup.find_all("button")
    keywords = ["submit", "envoyer", "valider", "sign up", "register", "s\'inscrire", "enregistrer", "s\'enregistrer"]
    for button in buttons:
        text = button.get_text(" ").lower()
        if any(keyword in text for keyword in keywords):
            return True
    return False


def has_placeholder_or_label(soup: BeautifulSoup) -> bool:
    inputs = soup.find_all("input")

    for field in inputs:
        if field.get("placeholder"):
            return True

        field_id = field.get("id")
        if field_id and soup.find("label", attrs={"for": field_id}):
            return True
    return False


def is_probable_form(soup: BeautifulSoup) -> bool:
    return (
        has_input_fields(soup)
        and (has_action_button(soup) or has_placeholder_or_label(soup))
    )

In [35]:
soup = BeautifulSoup(html, "lxml")

form_present = has_form_tag(soup)
input_present = has_input_fields(soup)

reasons: list[str] = []

if form_present:
    reasons.append("Balise HTML <form> detectée.")
if input_present:
    reasons.append("Champs input/select/textarea détecté.")

probable_form = False

if not form_present and is_probable_form(soup):
    probable_form = True
    reasons.append("Structure de type formulaire détectée à l'aide d'heuristiques HTML.")

dict_response = {
    "has_form": form_present,
    "forms_count": count_form_tags(soup),
    "has_inputs": input_present,
    "probable_form": probable_form,
    "reasons": reasons
    }

print(dict_response)

{'has_form': True, 'forms_count': 1, 'has_inputs': True, 'probable_form': False, 'reasons': ['Balise HTML <form> detectée.', 'Champs input/select/textarea détecté.']}
